In [ ]:
import tensorflow
import numpy as np
import tqdm
from pathlib import Path

import cv2

In [ ]:
base_path = Path('/media/fast/generated_images')
images = base_path / 'images'
denseposeout = base_path / 'densepose_out'

In [ ]:
for image_filename in tqdm.tqdm(images.iterdir()):
    if image_filename.suffix.lower() not in ('.png', '.jpg'):
        continue
    
    inds_mask_filename = denseposeout / f'{image_filename.stem}_INDS{image_filename.suffix}' # example '01111_INDS.png'
    iuv_mask_filename = denseposeout / f'{image_filename.stem}_IUV{image_filename.suffix}' # example '01111_IUV.png'
    
    # маска туловища
    image = cv2.imread(str(image_filename))
    iuv_mask = cv2.imread(str(iuv_mask_filename))[..., 0]
    body = (iuv_mask == 2).astype(np.uint8)[..., np.newaxis]
    image_body = image * body
    
    # обрежем туловище
    body_points = np.argwhere(body[..., 0] == 1)
    min_y, max_y = np.min(body_points[:, 0]), np.max(body_points[:, 0])
    min_x, max_x = np.min(body_points[:, 1]), np.max(body_points[:, 1])
    image_body_crop = image_body[min_y:max_y, min_x:max_x, :]
    
    cv2.imwrite(str(base_path / f'crops/body/{image_filename.name}'), image_body_crop)
    